## Task 1: Optimization model

In [1]:
import gurobipy as gp
from gurobipy import GRB

# Create optimization model for bidding strategy in DA and balancing market
# Probably good to write it as a class or function for easier repeated use

## Task 2: Data collection

Want to determine some features that we believe will be useful predictors of future wind power. Could be previous wind power, features of the weather, season or similar. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Import relevant data 


# Check correlation between features



# Scale data 
scaler = StandardScaler()
x = scaler.fit_transform(np.random.rand(100, 1))


# Split data into training and test set
split_fraction = 0.8
split_index = int(len(x) * split_fraction)
x_train, x_test = x[:split_index], x[split_index:]

## Task 3: Linear Regression

In [40]:
from sklearn.linear_model import LinearRegression

def gradient_descent(lr, x, y, init_guess = None, max_iter = 1000, tol = 1e-6):
    # Initialize theta
    if init_guess is None:
        theta = np.ones(x.shape[1])
    else:
        theta = init_guess
    
    N = len(x)
    t = 0
    converged = False
    while t < max_iter and not converged:
        new_theta = theta - lr * sum((theta.T @ x[i] - y[i]) * x[i] for i in range(N))
        t += 1
        if np.linalg.norm(new_theta - theta) < tol:
            converged = True
        theta = new_theta
    print(t)
    print(theta)
    return theta


# Implement gradient descent
theta_gd = ...
# Implement closed form solution
# theta_cfs = np.linalg.inv(x_train.T @ x_train) @ x_train.T @ y_train

# Compare the two solutions
print(theta_gd)
#print(theta_cfs)


# Evaluate model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Ellipsis


In [41]:
x = np.arange(0, 100)
y = 3 * x + 5 + np.random.normal(0, 50, len(x)) * 10
X = np.c_[np.ones(len(x)), x]
y = y.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

print(x_train.shape, y_train.shape)

theta_gd = gradient_descent(0.2, x_train, y_train, max_iter= 100)
print(theta_gd)
y_pred = x_test @ theta_gd
print(mean_squared_error(y_test, y_pred))

plt.plot(x, 3 * x + 5, label = 'True')
plt.plot(x_test[:,0], y_pred, label = 'Predicted')
plt.legend()
plt.show()

print(y)

(80, 2) (80, 1)
100
[nan nan]
[nan nan]


C:\Users\jacob\AppData\Local\Temp\ipykernel_3236\2597105109.py:14: RuntimeWarning: overflow encountered in add
  new_theta = theta - lr * sum((theta.T @ x[i] - y[i]) * x[i] for i in range(N))
C:\Users\jacob\AppData\Local\Temp\ipykernel_3236\2597105109.py:14: RuntimeWarning: invalid value encountered in matmul
  new_theta = theta - lr * sum((theta.T @ x[i] - y[i]) * x[i] for i in range(N))


ValueError: Input contains NaN.

## Task 4: Non-linear regression

In [ ]:
from sklearn.linear_model import PolynomialFeatures

## Task 5: Regularization

In [ ]:
from sklearn.linear_model import Ridge, Lasso


## Task 6: Revenue calculation for evaluation

## Task 7 (optional): Suggestion for improvement

In [ ]:
from sklearn.cluster import KMeans

## Task 8: Model 2